In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import seaborn as sb
sb.set_style('darkgrid')

from glob import glob
import matplotlib.pyplot as plt
import os
import random
import cv2

import gzip
import nibabel as nb

In [2]:
MriFiles = glob('/Volumes/DISK_IMG/10*/*T1w.nii.gz')
SosFiles = glob('/Volumes/DISK_IMG/10*/*Vp.nii.gz')

MriIm = [] # array of full images containing header and data of MRIs
MriHdr = [] # array of only the headers of MRIs
MriData = [] # array of only the data of MRIs

# similarly, for speed of sound maps
SosIm = []
SosHdr = []
SosData = []
for i in range(len(MriFiles)):
    MriIm.append(nb.load(MriFiles[i]))
    MriHdr.append(MriIm[i].header)
    MriData.append(MriIm[i].get_fdata())

    SosIm.append(nb.load(SosFiles[i]))
    SosHdr.append(SosIm[i].header)
    SosData.append(SosIm[i].get_fdata())

In [3]:
# put all the scans of all MRIs as members of a dataframe so that each cell of each row is a whole image
MriDf = pd.DataFrame()
for scan in range(len(MriData)):
    Mri = MriData[scan]
    for slice in range(len(Mri[1,1,:])):
        temp = {'img' : [Mri[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        MriDf = MriDf.append(tempDf)
print(MriDf.shape[0])

1600


In [5]:
# same for SoS
SosDf = pd.DataFrame()
for scan in range(len(SosData)):
    Sos = SosData[scan]
    for slice in range(len(Sos[1,1,:])):
        temp = {'img' : [Sos[:,:,slice]]}
        tempDf = pd.DataFrame.from_records(temp)
        SosDf = SosDf.append(tempDf)
print(SosDf.shape[0])

1600


In [6]:
# must vectorize each cell of the df (each (n,m) matrix) so structure is:
# X has form (n.m, 2) (col of 1s and col of values)
# y has form (n.m, 1)
vec_MriDf = pd.DataFrame()
for row in range(MriDf.shape[0]):
    temp = pd.DataFrame({'' : np.array([[MriDf.iloc[row][0].flatten()]]).tolist()})
    vec_MriDf = pd.concat([vec_MriDf,temp])


In [7]:
# same for Sos
vec_SosDf = pd.DataFrame()
for row in range(SosDf.shape[0]):
    temp = pd.DataFrame({'' : np.array([[SosDf.iloc[row][0].flatten()]]).tolist()})
    vec_SosDf = pd.concat([vec_SosDf,temp])

In [55]:
# we need to make a super dataframe with both cos we'll separate them to train the model but we want them to be divided the same way
# eg. if MRI#2 is in training, SoS#2 also has to be there
df = pd.concat([vec_MriDf, vec_SosDf], axis=1)
df.columns = ['MRI','SoS']
df.head()

,MRI,SoS
0,"[[32.05524826049805, 72.65767669677734, 19.232...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[32.053810119628906, 45.943233489990234, 26.7...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[38.461997985839844, 54.48716354370117, 16.02...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[19.22929573059082, 53.4140510559082, 41.6621...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[11.749947547912598, 24.567768096923828, 25.6...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."


In [56]:
# lets do 70% train, 15% test, 15% eval
df_train = pd.DataFrame()
train_indices = []
for i in range(int(0.7*df.shape[0])):
    # get 70% of the set for training from the whole thing randomly
    r = random.randint(0, df.shape[0]) 
    train_indices.append(r)
    df_train = df_train.append(df.iloc[r])
print(df_train.shape)
df_train.head()

(1120, 2)


,MRI,SoS
0,"[[4.641510009765625, 24.35289764404297, 17.378...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[45.77377700805664, 49.28479766845703, 50.442...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[59.653480529785156, 4.064392566680908, 29.77...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[21.418109893798828, 33.20078659057617, 49.27...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."
0,"[[63.84263610839844, 12.27415943145752, 14.723...","[[1480.0, 1480.0, 1480.0, 1480.0, 1480.0, 1480..."


In [66]:
# get the stuff that isnt in train
# dataframe1[~dataframe1.column1.isin(common.column1)]
df_test = pd.DataFrame()
test_indices = []
while df_test.shape[0] < 0.15*df.shape[0]:
    r = random.randint(0, df.shape[0]-1)

    if(r not in train_indices): 
        test_indices.append(r)
        df_test = df_test.append(df.iloc[r])
print("Number of test images: " + str(df_test.shape[0]))

df_eval = pd.DataFrame()
# for i in range(int(0.15*df.shape[0])):
while df_eval.shape[0] < 0.15*df.shape[0]:
    r = random.randint(0, df.shape[0]-1)

    if(r not in train_indices) and (r not in test_indices) :
        df_eval = df_eval.append(df.iloc[r])
print("Number of eval images: " + str(df_eval.shape[0]))


Number of test images: 240
Number of eval images: 240
